In [1]:
import DualwormFunctions as dw
import Observables as obs
%load_ext line_profiler
%load_ext memory_profiler
import RunBasis
import argparse
import StartStates as strst
import numpy as np

In [2]:
nst = 4;
nsm = 12;
nips = 20;
nrps = 1;
nb = 4;
measperiod = 1;
nt = 32
nh = 12

In [3]:
def genArgs():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('-f', type = str)

    parser.add_argument('--L', type = int, default = 12, help = 'Lattice side size')

    # COUPLINGS
    parser.add_argument('--J1', type = float, default = 1,
                        help = 'NN coupling') # nearest-neighbour coupling
    parser.add_argument('--J2', type = float, default = 0,
                        help = '2nd NN coupling') # 2nd NN coupling
    parser.add_argument('--J3', type = float, default = 0,
                        help = '3rd NN coupling') # 3rd NN coupling
    parser.add_argument('--J4', type = float, default = 0.0,
                        help = '4th NN coupling')
    parser.add_argument('--h', type = float, default = 1.0,
                        help = 'Magnetic field')    
    #NUMBER OF STEPS AND ITERATIONS
    parser.add_argument('--nst', type = int, default = nst,
                        help = 'number of thermalisation steps') # number of thermalisation steps
    parser.add_argument('--nsm', type = int, default = nsm,
                        help = 'number of measurements steps') # number of measurement steps
    parser.add_argument('--nips', type = int, default = nips,
                        help = 'number of worm constructions per MC step')
    parser.add_argument('--nrps', type = int, default = nrps,
                        help = 'number of replica loops per MC step')
    parser.add_argument('--measperiod', type = int, default = measperiod,
                        help = 'number of nips worm building + swaps between measurements')
    parser.add_argument('--nb', type = int, default = nb,
                        help = 'number of bins')

    #PARALLELISATION
    parser.add_argument('--ncores', type = int, default = 8,
                        help = 'number of threads to use')

    #WORM PARAMETERS
    parser.add_argument('--nmaxiter', type = int, default = 10,
                        help = '''maximal number of segments in a loop update over the
                        size of the lattice (1 = 1times the number of dualbonds in the
                        lattice)''')
    parser.add_argument('--randominit', default = False, action ='store_true',
                        help = 'intialise the states randomly')
    parser.add_argument('--same', default = False, action = 'store_true',
                        help = '''initialise all temperatures with the same
                        state (debug purposes)''')
    parser.add_argument('--magninit', default = True, action = 'store_true',
                        help = '''initialise all the temperature with the maximally magnetised GS''')
    parser.add_argument('--measupdate', default = False, action = 'store_true',
                       help = '''activate to mimic the action of the measuring tip''')
    parser.add_argument('--p', type = float, default = 0.0, 
                       help = '''prob of the measuring tip flipping the spin (number between 0 and 1)''')
    parser.add_argument('--ssf', default = True, action = 'store_true',
                        help = 'activate for single spin flip update')
    #TEMPERATURE PARAMETERS
    parser.add_argument('--t_list', nargs = '+', type = float, default = [0.05,60],
                        help = 'list of limiting temperature values')
    parser.add_argument('--nt_list', nargs = '+', type = int, default = [nt],
                        help = 'list of number of temperatures in between the given limiting temperatures')
    parser.add_argument('--log_tlist', default = True, action='store_true',
                        help = 'state whether you want the temperature be spaced log-like or linear-like (activate if you want log)')
    parser.add_argument('--stat_temps_lims', nargs = '+', type = float,
                        help = '''limiting temperatures for the various ranges of
                        measurements''') 
                        #default will be set to none, and then we can decide what to do later on.
    #MAGNETIC FIELD PARAMETERS
    parser.add_argument('--h_list', nargs = '+', default = [0.0, 1.0],type = float,
                        help = 'list of limiting magnetic field values')
    parser.add_argument('--nh_list', nargs = '+', default = [nh], type = int,
                        help = 'list of number of magnetic fields in between the given limiting temperatures')
    parser.add_argument('--stat_hfields_lims', nargs = '+', type = float,
                help = '''limiting magnetic fields for the various ranges of
                measurements''') 
    #CORRELATIONS PARAMETER
    parser.add_argument('--energy', default = True, action = 'store_true',
                        help = 'activate if you want to save the energy')
    parser.add_argument('--magnetisation', default =True, action = 'store_true',
                        help = 'activate if you want to save the magnetisation')
    parser.add_argument('--charges', default = False, action = 'store_true',
                        help = 'activate if you want to save the charges')
    parser.add_argument('--correlations', default = True, action = 'store_true',
                        help = 'activate if you want to save either central or all correlations')
    parser.add_argument('--all_correlations', default = False, action = 'store_true',
                        help = '''activate if you want to save the correlations for all non-equivalent
                        pairs of sites. Otherwise, will save central correlations.''')
    parser.add_argument('--firstcorrelations', default = True, action = 'store_true',
                        help = 'activate if you want to save first correlations, otherwise will save central')
    #SAVE
    parser.add_argument('--output', type = str, default = "testJ1hssf", help = 'saving filename (.pkl will be added)')
    args = parser.parse_args()
    return args

In [4]:
def callRunBasis():
    args = genArgs()
    return RunBasis.main(args)

In [5]:
%lprun -f dw.mcs_swaps meanstat, swapst, swapsh, failedupdatesth, failedupdates = callRunBasis() 

Lattice side size:  12
J1  1
J2  0
J3  0
J3st  0
Couplings extracted
Hamiltonian expression (without field) computed
Number of temperatures:  32
Temperatures: [5.00000000e-02 6.28490086e-02 7.89999577e-02 9.93013805e-02
 1.24819866e-01 1.56896097e-01 1.97215283e-01 2.47895701e-01
 3.11599981e-01 3.91674998e-01 4.92327707e-01 6.18846166e-01
 7.77877360e-01 9.77776419e-01 1.22904557e+00 1.54488592e+00
 1.94189096e+00 2.44091844e+00 3.06818608e+00 3.85664907e+00
 4.84773142e+00 6.09350227e+00 7.65941154e+00 9.62772844e+00
 1.21018638e+01 1.52118028e+01 1.91209345e+01 2.40346356e+01
 3.02110604e+01 3.79747039e+01 4.77334498e+01 6.00000000e+01]
Number of magnetic fields:  12
Magnetic fields:  [0.         0.09090909 0.18181818 0.27272727 0.36363636 0.45454545
 0.54545455 0.63636364 0.72727273 0.81818182 0.90909091 1.        ]
Fully random initialisation =  False
Identical initialisation =  False
Magnetisation initialisation =  True
Initialisation type:  magninit
Energies =  [[ -864.         

Time for mcsevolve = 0.3609930674235026
Time for tempering = 0.005012253920237223
Time for mapping to spins + computing statistics= 3.541157782077789
Energies =  [[ -864.          -896.18181818  -942.18181818  -981.81818182
  -1021.09090909 -1060.36363636 -1099.63636364 -1138.90909091
  -1178.18181818 -1217.45454545 -1256.72727273 -1296.        ]
 [ -864.          -892.72727273  -941.45454545  -981.81818182
  -1021.09090909 -1060.36363636 -1099.63636364 -1138.90909091
  -1178.18181818 -1217.45454545 -1256.72727273 -1296.        ]
 [ -864.          -885.81818182  -937.09090909  -980.72727273
  -1021.09090909 -1060.36363636 -1099.63636364 -1138.90909091
  -1178.18181818 -1217.45454545 -1256.72727273 -1296.        ]
 [ -864.          -884.54545455  -924.72727273  -976.36363636
  -1021.09090909 -1060.36363636 -1099.63636364 -1138.90909091
  -1178.18181818 -1217.45454545 -1256.72727273 -1296.        ]
 [ -864.          -879.27272727  -913.09090909  -964.90909091
  -1016.         -1060.36363

In [6]:
print(failedupdatesth/(nst*nips))

[[0.55297068 0.97040895 0.99887153 1.         1.         1.
  1.         1.         1.         1.         1.         1.        ]
 [0.55271991 0.94187886 0.99571759 0.99984568 0.99998071 1.
  1.         1.         1.         1.         1.         1.        ]
 [0.55414738 0.90710841 0.9876061  0.99887153 0.99990355 1.
  1.         1.         1.         1.         1.         1.        ]
 [0.55276813 0.86266397 0.97197145 0.99458912 0.99945023 0.99988426
  0.99996142 1.         1.         1.         1.         1.        ]
 [0.55478395 0.81936728 0.94563079 0.98420139 0.99633488 0.9992284
  0.9998939  0.99995177 1.         1.         1.         1.        ]
 [0.5501929  0.77562693 0.90645255 0.96361883 0.98858025 0.99679784
  0.99897762 0.99975887 0.99996142 0.99990355 1.         1.        ]
 [0.55242091 0.73992091 0.8630787  0.93715278 0.96997492 0.98732639
  0.99494599 0.99796489 0.9992091  0.99949846 0.99976852 0.99992284]
 [0.55240162 0.70653935 0.82032215 0.89819637 0.94360532 0.9701485

In [7]:
print(failedupdates/(nsm*nips*measperiod))

[[0.55229231 0.97082369 0.99903549 0.99996463 1.         1.
  1.         1.         1.         1.         1.         1.        ]
 [0.55474859 0.94396862 0.99590728 0.99975244 0.99998071 1.
  1.         1.         1.         1.         1.         1.        ]
 [0.5553948  0.907192   0.98852238 0.99862719 0.99983282 0.99998714
  1.         1.         1.         1.         1.         1.        ]
 [0.55413773 0.86437757 0.97208398 0.99505208 0.99910944 0.99983925
  0.99998071 1.         1.         1.         1.         1.        ]
 [0.5525045  0.82065329 0.94560185 0.98551633 0.99617091 0.99912551
  0.9998714  0.99995499 0.99998071 0.99999357 1.         1.        ]
 [0.55366191 0.77790316 0.90912423 0.96673418 0.98797261 0.99622557
  0.99883295 0.99965921 0.9998714  0.99995499 0.99996785 0.99998714]
 [0.55375514 0.73653549 0.86674383 0.9372653  0.97239583 0.98786651
  0.99509066 0.99780093 0.9991448  0.99971386 0.99987461 0.99994213]
 [0.55282922 0.70746528 0.82212256 0.89936021 0.94504565 

In [8]:
print(4*swapst/(nrps*nsm*measperiod*nh))

[0.83333333 0.75       0.72222222 0.61111111 0.55555556 0.38888889
 0.30555556 0.13888889 0.11111111 0.27777778 0.22222222 0.16666667
 0.08333333 0.02777778 0.05555556 0.02777778 0.05555556 0.05555556
 0.05555556 0.11111111 0.25       0.30555556 0.47222222 0.47222222
 0.63888889 0.58333333 0.5        0.66666667 0.75       0.91666667
 0.94444444 0.        ]


In [9]:
print(4*swapsh/(nrps*nsm*measperiod*nt))

[0.48958333 0.47916667 0.45833333 0.53125    0.55208333 0.61458333
 0.625      0.72916667 0.70833333 0.70833333 0.79166667 0.        ]
